In [16]:
%%writefile testutility.py
import pandas as pd
import logging
import os
import subprocess
import yaml
import gc
import re




Overwriting testutility.py


In [17]:
################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [18]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: bank-full
table_name: edsurv
inbound_delimiter: ";"
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    -age
    -job
    -marital
    -education
    -default
    -balance
    -housing
    -loan
    -contact
    -day
    -month
    -duration
    -campaign
    -pdays
    -previous
    -poutcome
    -y

Overwriting file.yaml


In [19]:
#Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")
config_data['inbound_delimiter']

#inspecting data of config file
config_data


#Reading file
df_pandas = pd.read_csv("bank-full.csv",delimiter=';')
df_pandas.head()



ModuleNotFoundError: No module named 'yaml'

In [ ]:
#validate the header of the file
util.col_header_val(df,config_data)
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine